In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
datagen=ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2

)

In [4]:
df="/kaggle/input/grapevine-leaves-image-dataset/Grapevine_Leaves_Image_Dataset"

In [5]:
train_data=datagen.flow_from_directory(
    df,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset="training"
)
val_data=datagen.flow_from_directory(
    df,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset="validation"
)

Found 400 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


# # Deep learning

In [34]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Input,Dropout,GlobalMaxPooling2D,LeakyReLU
from keras.models import Model
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping
from keras.applications import VGG16

In [7]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

# # m1

In [14]:
inputs1=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs1)
D=Dropout(0.5)(C)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
G=GlobalAveragePooling2D()(C)
D=Dense(32,activation="relu")(G)
outputs1=Dense(5,activation="softmax")(D)
model1=Model(inputs1,outputs1)
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model1.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,637 (84.52 KB)

 Trainable params: 21,637 (84.52 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model1.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727493879.276708     112 service.cc:145] XLA service 0x7fe3800057e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727493879.276773     112 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-09-28 03:24:46.367860: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,32,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,256,256]{3,2,1,0}, f32[32,64,256,256]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFil

 2/13 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.2188 - loss: 3.6272 

I0000 00:00:1727493888.105985     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.1910 - loss: 3.5054 - val_accuracy: 0.2000 - val_loss: 3.2458
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 431ms/step - accuracy: 0.1634 - loss: 3.2008 - val_accuracy: 0.2000 - val_loss: 3.0250
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 395ms/step - accuracy: 0.2075 - loss: 2.9782 - val_accuracy: 0.1800 - val_loss: 2.8478
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 428ms/step - accuracy: 0.2238 - loss: 2.8040 - val_accuracy: 0.2000 - val_loss: 2.6980
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 428ms/step - accuracy: 0.1521 - loss: 2.6656 - val_accuracy: 0.2000 - val_loss: 2.5762
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 422ms/step - accuracy: 0.2008 - loss: 2.4470 - val_accuracy: 0.2000 - val_loss: 2.3830
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 433ms/step - accuracy: 0.2172 - loss: 2.3612 - val_accuracy: 0.2000 - val_loss: 2.3115
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 412ms/step - accuracy: 0.1740 - loss: 2.2928 - val_accuracy: 0.2000 - val_loss: 2.

# # m2

In [21]:
inputs2=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs2)
D=Dropout(0.5)(C)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
C=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
C=Conv2D(256,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(C)
# M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
G=GlobalMaxPooling2D()(B)
D=Dense(32,activation="relu")(G)
outputs2=Dense(5,activation="softmax")(D)
model2=Model(inputs2,outputs2)
model2.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model2.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_4          │ (None, 256)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 398,597 (1.52 MB)

 Trainable params: 397,701 (1.52 MB)

 Non-trainable params: 896 (3.50 KB)

In [22]:
model2.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.2061 - loss: 16.9946 - val_accuracy: 0.2000 - val_loss: 13.1669
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 476ms/step - accuracy: 0.2284 - loss: 11.2600 - val_accuracy: 0.1700 - val_loss: 10.8762
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 432ms/step - accuracy: 0.2326 - loss: 10.8835 - val_accuracy: 0.2300 - val_loss: 10.5160
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 466ms/step - accuracy: 0.2636 - loss: 10.4806 - val_accuracy: 0.2000 - val_loss: 10.2319
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 429ms/step - accuracy: 0.3039 - loss: 10.1345 - val_accuracy: 0.2100 - val_loss: 9.9473
Epoch 5: early stopping


In [24]:
loss,accuracy=model2.evaluate(val_data)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 420ms/step - accuracy: 0.2615 - loss: 9.9186


# # m3

In [25]:
inputs3=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs3)
D=Dropout(0.5)(C)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
C=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
C=Conv2D(256,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(556,(5,5),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
# M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
G=GlobalMaxPooling2D()(B)
D=Dense(32,activation="relu")(G)
outputs3=Dense(5,activation="softmax")(D)
model3=Model(inputs3,outputs3)
model3.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model3.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 32, 32, 556)    │     3,558,956 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_5          │ (None, 556)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │        17,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,969,377 (15.14 MB)

 Trainable params: 3,967,369 (15.13 MB)

 Non-trainable params: 2,008 (7.84 KB)

In [27]:
model3.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 537ms/step - accuracy: 0.2354 - loss: 20.7285 - val_accuracy: 0.2000 - val_loss: 21.8410
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 433ms/step - accuracy: 0.1952 - loss: 19.8279 - val_accuracy: 0.2000 - val_loss: 19.9594
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 470ms/step - accuracy: 0.2333 - loss: 18.9618 - val_accuracy: 0.1800 - val_loss: 18.7406
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.1752 - loss: 18.1603 - val_accuracy: 0.2000 - val_loss: 17.8932
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 463ms/step - accuracy: 0.1802 - loss: 17.4151 - val_accuracy: 0.2000 - val_loss: 17.1786
Epoch 5: early stopping


In [28]:
loss,accuracy=model3.evaluate(val_data)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 397ms/step - accuracy: 0.2123 - loss: 17.1576


# # m4

In [31]:
inputs4=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs4)
D=Dropout(0.5)(C)

C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(256,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(556,(5,5),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
L=LeakyReLU(0.08)(C)

D=Dropout(0.5)(L)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(1056,(5,5),activation=None,padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
L=LeakyReLU(0.08)(C)
D=Dropout(0.5)(L)
B=BatchNormalization()(D)

G=GlobalMaxPooling2D()(B)

D=Dense(32,activation="relu")(G)
outputs4=Dense(5,activation="softmax")(D)

model4=Model(inputs4,outputs4)
model4.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model4.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 32, 32, 556)    │     3,558,956 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 32, 32, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 32, 32, 556)    │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 16, 16, 556)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 16, 16, 1056)   │    14,679,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16, 16, 1056)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 16, 16, 1056)   │             

 Total params: 18,669,057 (71.22 MB)

 Trainable params: 18,664,937 (71.20 MB)

 Non-trainable params: 4,120 (16.09 KB)

In [32]:
model4.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50


2024-09-28 03:46:40.918740: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[1056,556,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,556,16,16]{3,2,1,0}, f32[32,1056,16,16]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 03:46:44.455660: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 4.537098598s
Trying algorithm eng0{} for conv (f32[1056,556,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,556,16,16]{3,2,1,0}, f32[32,1056,16,16]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn

10/13 ━━━━━━━━━━━━━━━━━━━━ 1s 359ms/step - accuracy: 0.1978 - loss: 50.2718

2024-09-28 03:47:00.712803: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[1056,556,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,556,16,16]{3,2,1,0}, f32[16,1056,16,16]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-28 03:47:02.003162: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.290484199s
Trying algorithm eng0{} for conv (f32[1056,556,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,556,16,16]{3,2,1,0}, f32[16,1056,16,16]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn

13/13 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.2077 - loss: 49.3019 - val_accuracy: 0.2800 - val_loss: 46.3971
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 494ms/step - accuracy: 0.3028 - loss: 44.6207 - val_accuracy: 0.2000 - val_loss: 60.8474
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 504ms/step - accuracy: 0.3043 - loss: 41.9412 - val_accuracy: 0.2000 - val_loss: 39.2745
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 498ms/step - accuracy: 0.3532 - loss: 37.8534 - val_accuracy: 0.2000 - val_loss: 35.1706
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 495ms/step - accuracy: 0.3194 - loss: 33.8689 - val_accuracy: 0.2000 - val_loss: 31.5234
Epoch 5: early stopping


In [33]:
loss,accuracy=model4.evaluate(val_data)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 382ms/step - accuracy: 0.1696 - loss: 31.5995


# # m5

In [42]:
base_model=VGG16(weights="imagenet",include_top=False,input_shape=(256,256,3))
for layer in base_model.layers:
    layer.trainable =False
inti=base_model.get_layer("block5_conv3").output

C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inti)
D=Dropout(0.5)(C)

C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(128,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(256,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.5)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(556,(5,5),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
L=LeakyReLU(0.08)(C)

D=Dropout(0.5)(L)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(1056,(5,5),activation=None,padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
L=LeakyReLU(0.08)(C)
D=Dropout(0.5)(L)
B=BatchNormalization()(D)

G=GlobalMaxPooling2D()(B)

D=Dense(32,activation="relu")(G)
outputs5=Dense(5,activation="softmax")(D)

model5=Model(inputs=base_model.input,outputs=outputs5)
model5.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])
model5.summary()

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 16, 16, 32)     │       147,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 33,530,337 (127.91 MB)

 Trainable params: 18,811,529 (71.76 MB)

 Non-trainable params: 14,718,808 (56.15 MB)

In [43]:
model5.fit(train_data,epochs=50,validation_data=val_data,callbacks=[early_stopping])

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.2032 - loss: 35.9445 - val_accuracy: 0.2000 - val_loss: 20.1324
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 452ms/step - accuracy: 0.1707 - loss: 19.0749 - val_accuracy: 0.2000 - val_loss: 16.1084
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 446ms/step - accuracy: 0.2496 - loss: 16.0755 - val_accuracy: 0.2700 - val_loss: 15.0630
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 413ms/step - accuracy: 0.1887 - loss: 15.2060 - val_accuracy: 0.2000 - val_loss: 14.4536
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 430ms/step - accuracy: 0.2050 - loss: 14.4571 - val_accuracy: 0.2000 - val_loss: 13.9351
Epoch 5: early stopping
